# Google Form -> EDSL
This notebook provides example EDSL code for converting a non-EDSL survey into in an EDSL survey. This can be useful for accessing EDSL's built-in methods for analyzing survey data, and extending it with responses simulated with AI agents and diverse large language models.

[EDSL is an open-source library](https://github.com/expectedparrot/edsl) for simulating surveys, experiments and other research with AI agents and large language models. 
Before running the code below, please ensure that you have [installed the EDSL library](https://docs.expectedparrot.com/en/latest/installation.html) and either [activated remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) from your [Coop account](https://docs.expectedparrot.com/en/latest/coop.html) or [stored API keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for the language models that you want to use with EDSL. Please also see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started using EDSL.

## Designing the task as an EDSL survey
We design the task as an EDSL survey *about* the survey to be converted: a series of questions prompting a language model to read and reformat the contents of a given survey. The formatted responses of the language model are readily usable components of a new EDSL survey that can be administered to AI agents and/or human audiences.

## Creating a meta-survey
We start by selecting appropriate question types for reformatting the contents of a given survey.
[EDSL comes with many common question types](https://docs.expectedparrot.com/en/latest/questions.html) that we can choose from based on the form of the response that we want to get back from a model: multiple choice, checkbox, free text, linear scale, etc.

Here we use `QuestionList` to return information about all the questions in the survey at once, as a list. We create a sequence of questions, using the response to one question as an input to the next question. This step-wise approach can improve performance by allowing a model to focus on distinct tasks, and also allow us to pinpoint modifications to instructions as needed (some models will perform better and need fewer instructions than others). Note that we use a `{{ placeholder }}` for the text of the survey to be reformatted that we want to add to the initial question, which allows us to reuse it with other content (e.g., another survey):

In [1]:
from edsl import QuestionList

First we ask the model to return just the questions from the survey:

In [2]:
q1 = QuestionList(
    question_name="q_text",
    question_text="""
    You are being asked to extract questions from the text of a survey.
    Read the text and then return a list of all the questions in the 
    order that you find them. Return only the list of questions.
    Survey: {{ text }}
    """
)

Next we ask the model to format the questions as dictionaries, and specify the question text and type:

In [3]:
q2 = QuestionList(
    question_name="q_type",
    question_text="""
    Now create a dictionary for each question, using keys 'question_text' and 'question_type'.
    The value for 'question_text' is the question text you already identified.
    The value for 'question_type' should be the most appropriate of the following types:
    'multiple_choice', 'checkbox', 'linear_scale' or 'free_text'.
    Return only the list of dictionaries you have created, with the 2 key/value pairs for each question.
    """
)

Next we ask the model to add the question options (if any):

In [4]:
q3 = QuestionList(
    question_name="q_options",
    question_text="""
    Now add a key 'question_options' to each dictionary for all questions that are not free text,
    with a value that is a list of the answer options for the question.
    Preserve any integer options as integers, not strings.
    If there are labels for linear scale answer options then add another key 'option_labels'
    with a value that is a dictionary: the keys are the relevant integers and the values are the labels.
    Return only the list of dictionaries you have created with all relevant key/value pairs for each question.
    """
)

Finally, we ask the model to give each question a name:

In [5]:
q4 = QuestionList(
    question_name="q_name",
    question_text="""
    Now add a key 'question_name' to each dictionary.
    The value should be a unique short pythonic string.
    Return only the list of dictionaries that you have created, 
    with all the key/value pairs for each question.
    """
)

Next we combine the questions into a `Survey` in order to administer them together. 
We add a "memory" of each prior question in the survey so that the model will have the context and its answers on hand when answering each successive question:

In [6]:
from edsl import Survey

In [7]:
survey = Survey(questions = [q1, q2, q3, q4]).set_full_memory_mode()

## Adding content to questions
Next we create a `Scenario` object for the contents of a (non-EDSL) survey to be inserted in the first question. 
This allows us to reuse the questions with other content. [Learn more about using scenarios](https://docs.expectedparrot.com/en/latest/scenarios.html) to scale data labeling and other tasks.

Here we create a scenario for a [Google Form](https://forms.gle/GufjVVs5PfxUeyoj7) (a customer feedback survey) that we have stored as a publicly-accessible PDF at the [Coop](https://www.expectedparrot.com/explore).

Code for posting a PDF to the Coop (rerun with your own file):

In [8]:
from edsl import FileStore

fs = FileStore("customer_feedback_survey.pdf") # file type is automatically inferred
info = fs.push()
info

{'description': 'File: customer_feedback_survey.pdf',
 'object_type': 'scenario',
 'url': 'https://www.expectedparrot.com/content/eeb73f0d-c6fd-4e4f-8551-f6ead6d5750d',
 'uuid': 'eeb73f0d-c6fd-4e4f-8551-f6ead6d5750d',
 'version': '0.1.39.dev2',
 'visibility': 'unlisted'}

Retrieving a file (replace with UUID of any desired object at Coop):

In [9]:
pdf_file = FileStore.pull(info["uuid"])

Creating a scenario for the content:

In [10]:
from edsl import Scenario

In [11]:
s = Scenario({"text":pdf_file})

## Selecting language models
EDSL works with many popular language models that we can select to use in generating survey responses. 
You can provide your own API keys for models or activate remote inference to run surveys at the Expected Parrot server with any available models. 
[Learn more about working with language models](https://docs.expectedparrot.com/en/latest/language_models.html) and using [remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html).

In [12]:
from edsl import ModelList, Model

To see a list of all available models:

In [13]:
# Model.available()

Here we select several models to compare their responses:

In [14]:
models = ModelList(
    Model(m) for m in ["gemini-1.5-flash", "gpt-4o", "claude-3-5-sonnet-20240620"]
)

## Running a survey
Next we add the scenario and models to the survey and run it. 
This generates a dataset of `Results` that we can access with built-in methods for analysis. 
[Learn more about working with results](https://docs.expectedparrot.com/en/latest/results.html).

In [15]:
results = survey.by(s).by(models).run()

Job UUID,634ad26d-dd10-4d9b-bbbb-3917ec94771b
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/634ad26d-dd10-4d9b-bbbb-3917ec94771b
Error Report URL,https://www.expectedparrot.com/home/remote-inference/error/8d776101-4b08-460f-8826-8ccab571ffc4
Results UUID,dc50baae-a147-4213-abcb-78fab8dea081
Results URL,None


To see a list of all the components of the results that have been generated:

In [16]:
results.columns

,0
0,agent.agent_instruction
1,agent.agent_name
2,answer.q_name
3,answer.q_options
4,answer.q_text
5,answer.q_type
6,comment.q_name_comment
7,comment.q_options_comment
8,comment.q_text_comment
9,comment.q_type_comment


We can filter, sort, select and print components in a table:

In [17]:
(
    results
    .sort_by("model")
    .select("model", "q_name") #"q_text", "q_type", "q_options", "q_name")
)

,model.model,answer.q_name
0,claude-3-5-sonnet-20240620,"[{'question_text': 'How many times have you visited our website in the past month?', 'question_type': 'multiple_choice', 'question_options': [0, 1, 2, 3, 4, '5 or more'], 'question_name': 'visit_frequency'}, {'question_text': 'How easy was it to find the information you were looking for?', 'question_type': 'linear_scale', 'question_options': [1, 2, 3, 4, 5], 'option_labels': {'1': 'Very difficult', '5': 'Very easy'}, 'question_name': 'info_findability'}, {'question_text': 'How would you rate the overall design and layout of our website?', 'question_type': 'linear_scale', 'question_options': [1, 2, 3, 4, 5], 'option_labels': {'1': 'Poor', '5': 'Excellent'}, 'question_name': 'design_rating'}, {'question_text': 'Did you encounter any technical issues while using our website?', 'question_type': 'multiple_choice', 'question_options': ['Yes', 'No'], 'question_name': 'technical_issues'}, {'question_text': 'How likely are you to recommend our website to others?', 'question_type': 'linear_scale', 'question_options': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'option_labels': {'0': 'Not at all likely', '10': 'Extremely likely'}, 'question_name': 'recommendation_likelihood'}, {'question_text': 'What is your primary reason for visiting our website?', 'question_type': 'multiple_choice', 'question_options': ['Information seeking', 'Product purchase', 'Customer support', 'General browsing', 'Other'], 'question_name': 'visit_reason'}, {'question_text': 'How satisfied are you with the content available on our website?', 'question_type': 'linear_scale', 'question_options': [1, 2, 3, 4, 5], 'option_labels': {'1': 'Very dissatisfied', '5': 'Very satisfied'}, 'question_name': 'content_satisfaction'}, {'question_text': ""How does our website compare to similar websites you've used?"", 'question_type': 'multiple_choice', 'question_options': ['Much worse', 'Somewhat worse', 'About the same', 'Somewhat better', 'Much better'], 'question_name': 'website_comparison'}, {'question_text': 'What additional features or information would you like to see on our website?', 'question_type': 'free_text', 'question_name': 'desired_features'}, {'question_text': 'Do you have any other comments or suggestions for improving our website?', 'question_type': 'free_text', 'question_name': 'additional_feedback'}]"
1,gemini-1.5-flash,"[{'question_text': 'Email', 'question_type': 'free_text', 'question_name': 'email'}, {'question_text': 'How did you first hear about our company?', 'question_type': 'free_text', 'question_name': 'hear_about'}, {'question_text': 'Which of the following services have you used?', 'question_type': 'checkbox', 'question_options': ['service1', 'service2', 'service3'], 'question_name': 'used_services'}, {'question_text': 'On a scale from 1 to 5, how satisfied are you with our customer service?', 'question_type': 'linear_scale', 'question_options': [1, 2, 3, 4, 5], 'option_labels': {'1': 'Very dissatisfied', '2': 'Dissatisfied', '3': 'Neutral', '4': 'Satisfied', '5': 'Very satisfied'}, 'question_name': 'customer_satisfaction'}, {'question_text': 'How many times have you purchased from us in the past year?', 'question_type': 'free_text', 'question_name': 'purchase_frequency'}, {'question_text': 'Please provide any additional comments or suggestions you have.', 'question_type': 'free_text', 'question_name': 'comments'}]"
2,gpt-4o,nan


## Creating a new EDSL survey
Now we can construct a new EDSL survey with the reformatted components of the original survey.
This is done by creating `Question` objects with the question components, passing them to a new `Survey`, and then optionally designing and assigning AI agents to answer the survey.

Here we select one of the model's responses to use:

In [18]:
from edsl import Question

In [19]:
questions_list = results.filter("model.model == 'claude-3-5-sonnet-20240620'").select("q_name").to_list()[0]
questions_list

[{'question_text': 'How many times have you visited our website in the past month?',
  'question_type': 'multiple_choice',
  'question_options': [0, 1, 2, 3, 4, '5 or more'],
  'question_name': 'visit_frequency'},
 {'question_text': 'How easy was it to find the information you were looking for?',
  'question_type': 'linear_scale',
  'question_options': [1, 2, 3, 4, 5],
  'option_labels': {'1': 'Very difficult', '5': 'Very easy'},
  'question_name': 'info_findability'},
 {'question_text': 'How would you rate the overall design and layout of our website?',
  'question_type': 'linear_scale',
  'question_options': [1, 2, 3, 4, 5],
  'option_labels': {'1': 'Poor', '5': 'Excellent'},
  'question_name': 'design_rating'},
 {'question_text': 'Did you encounter any technical issues while using our website?',
  'question_type': 'multiple_choice',
  'question_options': ['Yes', 'No'],
  'question_name': 'technical_issues'},
 {'question_text': 'How likely are you to recommend our website to others?'

In [20]:
edsl_questions = [Question(**q) for q in questions_list]
edsl_questions

[Question('multiple_choice', question_name = """visit_frequency""", question_text = """How many times have you visited our website in the past month?""", question_options = [0, 1, 2, 3, 4, '5 or more']),
 Question('linear_scale', question_name = """info_findability""", question_text = """How easy was it to find the information you were looking for?""", question_options = [1, 2, 3, 4, 5], option_labels = {1: 'Very difficult', 5: 'Very easy'}),
 Question('linear_scale', question_name = """design_rating""", question_text = """How would you rate the overall design and layout of our website?""", question_options = [1, 2, 3, 4, 5], option_labels = {1: 'Poor', 5: 'Excellent'}),
 Question('multiple_choice', question_name = """technical_issues""", question_text = """Did you encounter any technical issues while using our website?""", question_options = ['Yes', 'No']),
 Question('linear_scale', question_name = """recommendation_likelihood""", question_text = """How likely are you to recommend our

In [21]:
new_survey = Survey(edsl_questions)

We can inspect the survey that has been created:

In [22]:
new_survey

,option_labels,question_type,question_name,question_text,question_options
0,nan,multiple_choice,visit_frequency,How many times have you visited our website in the past month?,"[0, 1, 2, 3, 4, '5 or more']"
1,"{1: 'Very difficult', 5: 'Very easy'}",linear_scale,info_findability,How easy was it to find the information you were looking for?,"[1, 2, 3, 4, 5]"
2,"{1: 'Poor', 5: 'Excellent'}",linear_scale,design_rating,How would you rate the overall design and layout of our website?,"[1, 2, 3, 4, 5]"
3,nan,multiple_choice,technical_issues,Did you encounter any technical issues while using our website?,"['Yes', 'No']"
4,"{0: 'Not at all likely', 10: 'Extremely likely'}",linear_scale,recommendation_likelihood,How likely are you to recommend our website to others?,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]"
5,nan,multiple_choice,visit_reason,What is your primary reason for visiting our website?,"['Information seeking', 'Product purchase', 'Customer support', 'General browsing', 'Other']"
6,"{1: 'Very dissatisfied', 5: 'Very satisfied'}",linear_scale,content_satisfaction,How satisfied are you with the content available on our website?,"[1, 2, 3, 4, 5]"
7,nan,multiple_choice,website_comparison,How does our website compare to similar websites you've used?,"['Much worse', 'Somewhat worse', 'About the same', 'Somewhat better', 'Much better']"
8,nan,free_text,desired_features,What additional features or information would you like to see on our website?,nan
9,nan,free_text,additional_feedback,Do you have any other comments or suggestions for improving our website?,nan


## Designing AI agents
EDSL comes with methods for designing AI agent personas for language models to use in answering questions.
An `Agent` is created by passing a dictionary of relevant `traits`. It can then be assigned to a survey using the `by()` method when the survey is run (the same as we do with scenarios and models).

We can import existing data to create agents representing audiences of interest, or use EDSL to generate personas:

In [23]:
q_personas = QuestionList(
    question_name="personas",
    question_text="Draft 5 diverse personas for customers of a landscape business in New England capable of answering a feedback survey."
)

If we do not specify a model to use in running the question, the default model GPT 4 preview is used:

In [24]:
personas = q_personas.run().select("personas").to_list()[0]
personas

Job UUID,924b27e8-246f-4150-aad6-fe330babfb51
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/924b27e8-246f-4150-aad6-fe330babfb51
Error Report URL,None
Results UUID,c932353c-a2e8-45e5-9b76-f3b1c48476c9
Results URL,None


['Retired couple looking to enhance their garden for leisure',
 'Young family seeking a safe play area for children',
 'Eco-conscious homeowner interested in sustainable landscaping',
 'Busy professional wanting low-maintenance yard solutions',
 'Small business owner needing attractive commercial property landscaping']

Note that the personas can be (much) longer and include key/value pairs for any desired traits; we keep it simple here for demonstration purposes.
Here we pass the personas to a list of agents and have them answer the survey:

In [25]:
from edsl import AgentList, Agent

In [26]:
agents = AgentList(
    Agent(
        traits = {"persona":p},
        instruction = """
        You are answering a customer feedback survey for a landscaping business that you have engaged in the past.
        Your answers are completely confidential.
        """
    )
    for p in personas
)

In [27]:
new_results = new_survey.by(agents).by(models).run()

Job UUID,076e2692-53da-4493-83fa-53ef51e6ea8a
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/076e2692-53da-4493-83fa-53ef51e6ea8a
Error Report URL,None
Results UUID,24f26640-029f-4d02-ab61-3fd7fa3db457
Results URL,None


In [28]:
(
    new_results
    .sort_by("model", "persona")
    .select("model", "persona", "answer.*")
)

,model.model,agent.persona,answer.technical_issues,answer.visit_reason,answer.recommendation_likelihood,answer.website_comparison,answer.info_findability,answer.visit_frequency,answer.desired_features,answer.additional_feedback,answer.design_rating,answer.content_satisfaction
0,claude-3-5-sonnet-20240620,Busy professional wanting low-maintenance yard solutions,No,Information seeking,7,Somewhat better,4,1,"As a busy professional looking for low-maintenance yard solutions, I would appreciate seeing the following additional features or information on your website: 1. A dedicated section for low-maintenance landscaping options and designs. 2. Detailed information about drought-resistant and native plants that require minimal care. 3. Time estimates for various services, so I can better plan around my busy schedule. 4. A gallery of before-and-after photos showcasing low-maintenance yard transformations. 5. Maintenance schedules and tips for different types of low-maintenance landscapes. 6. Information about automated irrigation systems and smart landscaping technology. 7. A blog with quick, easy yard care tips for busy homeowners. 8. Pricing guides or packages specifically tailored for low-maintenance options. 9. Customer testimonials from other busy professionals who have used your services. 10. An easy-to-use online booking system for consultations or services.","As a busy professional, I appreciate websites that are streamlined and easy to navigate. I would suggest making sure your site has clear sections for different services, especially highlighting low-maintenance options. It would be great to see a gallery of before/after photos to quickly visualize potential results. Also, an online scheduling tool for consultations would be very convenient for those of us with packed schedules. Lastly, perhaps include some tips or a blog section about maintaining a beautiful yard with minimal time investment - that kind of content would be really valuable for someone in my position.",3,4
1,claude-3-5-sonnet-20240620,Eco-conscious homeowner interested in sustainable landscaping,No,Information seeking,8,Somewhat better,4,2,"As an eco-conscious homeowner interested in sustainable landscaping, I would love to see the following additional features or information on your website: 1. A dedicated section on sustainable landscaping practices and how your company implements them. 2. Information about native plant species you use and their benefits for local ecosystems. 3. Details on water-saving techniques and drought-resistant landscaping options. 4. A showcase of eco-friendly materials used in hardscaping, such as permeable pavers or recycled materials. 5. Information on organic lawn care and pest management methods you employ. 6. Case studies or before-and-after examples of sustainable landscaping projects you've completed. 7. A blog or resources section with tips for homeowners on maintaining an eco-friendly landscape. 8. Information about any certifications or partnerships your company has related to environmental stewardship. 9. Details on how you minimize waste and practice responsible disposal methods. 10. A calculator or tool to help estimate the environmental impact and potential savings of sustainable landscaping choices.","As an eco-conscious homeowner interested in sustainable landscaping, I would suggest adding more content and resources related to environmentally-friendly landscaping practices. It would be great to see: 1. A dedicated section on sustainable landscaping techniques, such as xeriscaping, native plant gardening, and water conservation methods. 2. Information about organic lawn care and pesticide-free gardening alternatives. 3. A guide to selecting native plants that support local ecosystems and require less maintenance. 4. Tips for creating wildlife-friendly gardens that attract pollinators and birds. 5. Resources on composting and natural soil improvement techniques. 6. Showcase examples of successful sustainable landsc

## Posting to the Coop
The [Coop](https://www.expectedparrot.com/content/explore) is a platform for creating, storing and sharing LLM-based research.
It is fully integrated with EDSL and accessible from your workspace or Coop account page.
Learn more about [creating an account](https://www.expectedparrot.com/login) and [using the Coop](https://docs.expectedparrot.com/en/latest/coop.html).

Here we demonstrate how to post this notebook:

In [29]:
from edsl import Notebook

In [30]:
n = Notebook(path = "google_form_to_edsl.ipynb")

In [31]:
info = n.push(description = "Example code for using EDSL to convert a non-EDSL survey into EDSL", visibility = "public")
info

{'description': 'Example code for using EDSL to convert a non-EDSL survey into EDSL',
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/3de426ce-e5a4-4434-ad01-3d91eb0e183b',
 'uuid': '3de426ce-e5a4-4434-ad01-3d91eb0e183b',
 'version': '0.1.39.dev2',
 'visibility': 'public'}

Updating an opject at the Coop:

In [32]:
n = Notebook(path = "google_form_to_edsl.ipynb") # resave

In [33]:
n.patch(uuid = info["uuid"], value = n)

{'status': 'success'}